**IMPORTAR LIBRERIAS**

---

In [46]:
# Librerias de manejo de datos
import pandas as pd

In [47]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()


✅ Conexión exitosa a la base de datos


In [48]:
# Importar librerías para la conexión a la base de datos con SQLAlchemy
from db_connection import get_engine

engine = get_engine()

In [49]:
# 📌 Obtener todas las tablas
cursor.execute("SHOW TABLES;")
tablas = [fila[0] for fila in cursor.fetchall()]
print(f"Total de tablas encontradas: {len(tablas)}")


Total de tablas encontradas: 50


---

**FUNCIONES**

---

In [50]:
def borrar_lista_tablas(tablas_a_borrar):
    """
    Borra las tablas de la base de datos que se pasan como argumento.
    """
    for tabla in tablas_a_borrar:
        try:
            cursor.execute(f"DROP TABLE `{tabla}`;")
            tablas.remove(tabla)
            print(f"✅ Tabla eliminada: {tabla}")
        except Exception as e:
            print(f"❌ Error al eliminar {tabla}: {e}")
    
    # Confirmar cambios
    conexion.commit()

------

**FILTROS PARA SEGREGAR Y ELIMINAR LAS TABLAS QUE NO SON ÚTILES**


---

*PRIMER FILTRO: Tablas con menos de 2 registros*

In [51]:
# Escoger tablas a eliminar
tablas_a_borrar = []

for tabla in tablas:
    try:
        cursor.execute(f"SELECT COUNT(*) FROM `{tabla}`;")
        total = cursor.fetchone()[0]
        if total <= 1:
            tablas_a_borrar.append(tabla)
    except Exception as e:
        print(f"❌ Error evaluando tabla {tabla}: {e}")

print(f"\nTablas candidatas para eliminación ({len(tablas_a_borrar)}):")
print(tablas_a_borrar)



Tablas candidatas para eliminación (0):
[]


In [53]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS CANDIDATAS

# Desactivar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

borrar_lista_tablas(tablas_a_borrar)

# Volver a activar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")



---

*SEGUNDO FILTRO: Tablas posiblemente innecesarias por el nombre y breve análisis*

In [54]:
# Imprimir los nombres de las tablas restantes
for tabla in tablas:
    print(tabla)


accounting_account_balances
accounting_accounts
accounting_movements
accounting_voucher_items
accounting_voucher_types
accounting_vouchers
billing_numberings
contact_relationships
contact_statements
contacts
contract_salary_history
costs_and_expenses
costs_and_expenses_categories
coupons
document_items
documents
ecommerce_legal_info
ecommerce_purchase_orders
employee_contracts
employee_positions
employees
fixed_asset_depreciations
fixed_assets
fixed_assets_groups
headquarter_warehouses
headquarters
inventory_adjustments
inventory_groups
item_balance
item_categories
item_depreciations
item_kardex
item_subcategories
item_variations
items
payment_conditions
payments
payroll_deductions
payroll_details
payroll_incomes
payroll_providers
payrolls
price_lists
retention_concepts
retentions
retentions_applied
retentions_certificates
taxes
warehouse_transfer_logs
warehouses


*ANÁLISIS Y CONCLUSIONES DE LA INFORMACIÓN OBTENIDA*

**Tablas clave para análisis de gestión empresarial (💡 Mantener)**

Estas son probablemente las más útiles para análisis de KPIs, dashboards o visualización de negocio:

*Categoría	//  Tablas clave*

📦 Inventario	= items, item_balance, item_kardex, warehouses, inventory_adjustments, item_categories, item_subcategories, item_variations

💼 Ventas / Documentos	= documents, document_items, contacts (clientes/proveedores)

🧾 Contabilidad	= accounting_vouchers, accounting_voucher_items, accounting_accounts, accounting_movements, retentions, retention_concepts, retentions_applied, accounting_account_balances

👥 Empleados / Nómina	= employees, employee_contracts, payrolls, payroll_details, payroll_incomes, payroll_deductions

🛒 Compras	= ecommerce_purchase_orders

📊 Precios	= price_lists

🏪 Organización	= headquarters, user_headquarters, roles, user_roles, company_areas

🧾 Impuestos	= taxes

🏭 Activos fijos	= fixed_assets, fixed_asset_depreciations, fixed_assets_groups


**Relevancia baja o dependiente del contexto (🤔 Revisar)**

*Estas podrían ser útiles dependiendo del foco del proyecto (clientes, pedidos, legal, e-commerce):*

contact_relationships, contact_accounts, contact_statements

coupons, coupon_groups (si hay ecommerce)

employee_positions, contract_salary_history

plan_electronic_documents, plan_system_controller

ecommerce_legal_info

costs_and_expenses, costs_and_expenses_categories

radian_documents (puede ser de facturación electrónica)

dining_tables (¿restaurante? puede ser irrelevante)

system_counters, system_restrictions

In [55]:
# Lista de tablas candidatas a eliminar
tablas_candidatas = [
    "api_access_tokens",
    "electronic_payroll_test_set",
    "migrations",
    "oauth_clients",
    "paynilo",
    "role_permissions",
    "template_versions",
    "templates",
    "user_data",
    "values_x_item",
    "warehouse_transfer_logs",
    "billing_numberings",
    "custom_fields",
    "configurations",
    "system_counters",
    "system_restrictions"
]

---

*TERCER FILTRO: tablas a eliminar según sus columnas y número de filas*

In [56]:

# Mostrar información muy básica de las tablas candidatas
for tabla in tablas_candidatas:
    try:
        df = pd.read_sql(f"SELECT * FROM `{tabla}` LIMIT 5", engine)
        print(f"\n📘 Tabla: {tabla}")
        print("Columnas:", df.columns.tolist())
        print("size: ", df.size)
    except Exception as e:
        print(f"\n❌ Error al cargar tabla {tabla}: {e}")


❌ Error al cargar tabla api_access_tokens: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'novateam.api_access_tokens' doesn't exist
[SQL: SELECT * FROM `api_access_tokens` LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

❌ Error al cargar tabla electronic_payroll_test_set: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'novateam.electronic_payroll_test_set' doesn't exist
[SQL: SELECT * FROM `electronic_payroll_test_set` LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

❌ Error al cargar tabla migrations: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'novateam.migrations' doesn't exist
[SQL: SELECT * FROM `migrations` LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

❌ Error al cargar tabla oauth_clients: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'novateam.oauth_clients' doesn't exist
[SQL: SELECT * FROM `oauth_clients` LIMIT 5]
(Background on this error at

**Tablas candidatas fuertes para eliminación (por contenido y nombre)**

Estas tablas tienen pocos registros, estructuras irrelevantes para KPIs o funciones accesorias que no impactan directamente en la gestión:

*Tabla	// Motivo principal*

api_access_tokens	Manejo técnico de tokens

electronic_payroll_test_set	Datos de prueba (test set)

migrations	Solo para migraciones del sistema

oauth_clients	Seguridad/autenticación interna

paynilo	Integración de pagos externa

role_permissions	Control de permisos internos

template_versions	Versiones de plantillas, sin datos

templates	Estructura visual, no analítica

user_data	Solo datos de área/cargo del usuario

custom_fields	Campos personalizados (metadatos)

system_counters	Estadísticas internas del sistema

system_restrictions	Restricciones de uso/licencia

values_x_item	Datos muy técnicos por item, posiblemente redundantes

warehouse_transfer_logs	Registro técnico, no necesariamente útil para KPIs

----

*CUARTO FILTRO: Investigar mas a fondo las tablas dudosas*

**⚠️ Tablas dudosas (verificar con .info() antes de decidir)**

Estas tienen algo más de contenido o potencial analítico dependiendo del negocio:

*billing_numberings:* Tiene 155 registros. Podría contener información útil para facturación si los números se asocian a documentos.

*configurations:* Solo 25 registros, pero hay que ver si son configuraciones claves que afectan reportes.



In [57]:
# Obtener el esquema de cada tabla para decidir si son candidatas a eliminación
tablas_dudosas = ['billing_numberings', 'configurations'] # Agregar mas tablas si es necesario

for tabla in tablas_dudosas:
    print(f"\n📋 Tabla: {tabla}")
    try:
        df = pd.read_sql(f"SELECT * FROM {tabla}", con=engine)
        print("▶️ .info():")
        print(df.info())
        print("▶️ Primeras filas:")
        display(df.head())
    except Exception as e:
        print(f"❌ Error al cargar {tabla}: {e}")



📋 Tabla: billing_numberings
▶️ .info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      50 non-null     int64         
 1   document_type_id        50 non-null     int64         
 2   pos_numbering           50 non-null     int64         
 3   name                    50 non-null     object        
 4   prefix                  50 non-null     object        
 5   initial_number          50 non-null     int64         
 6   current_number          50 non-null     int64         
 7   final_number            22 non-null     float64       
 8   valid_since             22 non-null     object        
 9   valid_until             22 non-null     object        
 10  validity                22 non-null     float64       
 11  preferred               50 non-null     int64         
 12  electronic_

,id,document_type_id,pos_numbering,name,prefix,initial_number,current_number,final_number,valid_since,valid_until,...,factible_report_id,template,headquarter_id,warehouse_id,status,user_id,deleted_at,created_at,updated_at,contingency
0,1,11,0,Remisiones,,0,56,NaN,None,None,...,None,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,None,NaT,2024-05-24 09:54:00,0
1,2,12,0,Cotizaciones,,0,91,NaN,None,None,...,None,"{""id"": 3, ""name"": ""POS"", ""template_type"": ""1""}",1,1,1,10,None,NaT,2024-07-13 11:53:24,0
2,3,13,0,Recibos de Caja,,0,506,NaN,None,None,...,None,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,None,NaT,2024-07-17 10:58:41,0
3,4,14,0,Comprobantes de Pago,,0,113,NaN,None,None,...,None,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,None,NaT,2024-07-17 11:00:31,0
4,5,15,0,Órdenes de Compra,,0,23,NaN,None,None,...,None,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,None,NaT,2024-04-03 16:10:15,0



📋 Tabla: configurations
❌ Error al cargar configurations: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'novateam.configurations' doesn't exist
[SQL: SELECT * FROM configurations]
(Background on this error at: https://sqlalche.me/e/20/f405)


---

**TABLAS FINALES A ELIMINAR DESPUÉS DE TODOS LOS FILTROS Y ANÁLISIS INDIVIDUAL**

Falta escoger y seleccionar individualmente 

In [58]:
tablas_a_eliminar = [
    "api_access_tokens",             # Manejo técnico de tokens
    "electronic_payroll_test_set",   # Datos de prueba (test set)
    "migrations",                    # Solo para migraciones del sistema
    "oauth_clients",                 # Seguridad/autenticación interna
    "paynilo",                       # Integración de pagos externa
    "role_permissions",             # Control de permisos internos
    "template_versions",            # Versiones de plantillas, sin datos
    "templates",                    # Estructura visual, no analítica
    "user_data",                    # Solo datos de área/cargo del usuario
    "custom_fields",                # Campos personalizados (metadatos)
    "system_counters",              # Estadísticas internas del sistema
    "system_restrictions",          # Restricciones de uso/licencia
    "values_x_item",                # Datos técnicos por ítem
    "user_roles",
    "user_data",
    "user_headquarters",
    "term_and_conditions,",
    "severance_payments",
    "roles", 
    "role_permissions",
    "radian_events",
    "radian_documents",
    "plan_system_controller",
    "plan_electronic_documents",
    "api_access_tokens",
    "configurations",
    "company_areas",
    "consolidated_retention_certificates",
    "contact_accounts",
    "contact_relationship",
    "contact_statement",
    "contact",
    "custom_fields",
    "dining_tables",
    "ecomerce_legal_info",
    "ecomerce_purchase_orders",
    "electronic_payroll_test_set", 
    "electronic_contracts",
    "migraciones ]"     # Registro técnico, no útil para KPIs

]




In [59]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS FINALES ¡OJO HAY QUE ESCOGER ANTES DE EJECUTAR!

# Desactivar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

borrar_lista_tablas(tablas_a_eliminar)

# Volver a activar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")


❌ Error al eliminar api_access_tokens: 1051 (42S02): Unknown table 'novateam.api_access_tokens'
❌ Error al eliminar electronic_payroll_test_set: 1051 (42S02): Unknown table 'novateam.electronic_payroll_test_set'
❌ Error al eliminar migrations: 1051 (42S02): Unknown table 'novateam.migrations'
❌ Error al eliminar oauth_clients: 1051 (42S02): Unknown table 'novateam.oauth_clients'
❌ Error al eliminar paynilo: 1051 (42S02): Unknown table 'novateam.paynilo'
❌ Error al eliminar role_permissions: 1051 (42S02): Unknown table 'novateam.role_permissions'
❌ Error al eliminar template_versions: 1051 (42S02): Unknown table 'novateam.template_versions'
❌ Error al eliminar templates: 1051 (42S02): Unknown table 'novateam.templates'
❌ Error al eliminar user_data: 1051 (42S02): Unknown table 'novateam.user_data'
❌ Error al eliminar custom_fields: 1051 (42S02): Unknown table 'novateam.custom_fields'
❌ Error al eliminar system_counters: 1051 (42S02): Unknown table 'novateam.system_counters'
❌ Error al e

**TABLAS ASIGNADAS AL ANÁLISIS DE RH - ALEXANDRA**
A continuación, se irá explorando una a una, las tablas y su contenido para visualizar qué tan viables son para el ejercicio propuesto, se realizará limpieza de datos, se eliminarán nulls según convenga o se reemplazarán por n/a.  

employee_positions, employees, , employee_contracts, contract_salary_history, payrolls, payroll_details, payroll_incomes, payroll_deductions, payroll_providers, costs_and_expenses, costs_and_expenses_categories, headquarters, coupons. 



In [60]:
#TABLA 1: EMPLOYEE_POSITIONS  O Posiciones de empleados. 
# Script para obtener la información de las columnas de 'employee_positions'
try:
    # Obtener la descripción de las columnas de la tabla 'employee_positions'
    cursor.execute("DESCRIBE employee_positions;")
    columnas_employee_positions = cursor.fetchall()

    print("Columnas de la tabla 'employee_positions':")
    for columna in columnas_employee_positions:
        # Cada 'columna' es una tupla con (nombre_columna, tipo_dato, ...)
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Ocurrió un error al intentar obtener las columnas de 'employee_positions': {e}")

Columnas de la tabla 'employee_positions':
  - id (int unsigned)
  - name (varchar(255))
  - description (varchar(500))
  - salary (double(16,2))
  - status (tinyint(1))
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)


Descripción relacionada a las columnas:
id: int unsigned (Identificador único de la posición)
name: varchar(255) (Nombre de la posición)
description: varchar(500) (Descripción de la posición)
salary: double(16,2) (Salario asociado a la posición)
status: tinyint(1) (Estado de la posición, probablemente 1 para activa, 0 para inactiva)
user_id: int unsigned (ID del usuario que creó o modificó el registro, parece una clave foránea a una tabla de usuarios)
created_at: timestamp (Marca de tiempo de creación del registro)
updated_at: timestamp (Marca de tiempo de la última actualización del registro)

In [61]:
# Script para la verificación inicial de datos en 'employee_positions'
try:
    # 1. Obtener el número total de registros
    cursor.execute("SELECT COUNT(*) FROM employee_positions;")
    total_registros = cursor.fetchone()[0]
    print(f"\nTotal de registros en 'employee_positions': {total_registros}")

    # 2. Verificar valores nulos por columna
    print("\nValores nulos por columna en 'employee_positions':")
    for columna_info in columnas_employee_positions: # Usamos 'columnas_employee_positions' que ya obtuvimos
        nombre_columna = columna_info[0]
        cursor.execute(f"SELECT COUNT(*) FROM employee_positions WHERE {nombre_columna} IS NULL;")
        nulos_count = cursor.fetchone()[0]
        if nulos_count > 0:
            print(f"  - {nombre_columna}: {nulos_count} nulos")
        else:
            print(f"  - {nombre_columna}: 0 nulos")

    # 3. Verificar filas duplicadas (basado en todas las columnas, o solo el ID si es la clave primaria)
    # Asumimos que 'id' debería ser único
    cursor.execute("SELECT COUNT(id) - COUNT(DISTINCT id) FROM employee_positions;")
    duplicados_id = cursor.fetchone()[0]
    print(f"\nFilas duplicadas basadas en 'id' en 'employee_positions': {duplicados_id}")

    
    # 4. Obtener valores únicos para columnas específicas
    print("\nValores únicos en columnas clave de 'employee_positions':")

    # Para 'name'
    cursor.execute("SELECT DISTINCT name FROM employee_positions LIMIT 10;") # Limitar para no cargar demasiados si hay muchos
    nombres_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - name (primeros 10 únicos): {nombres_unicos}")
    cursor.execute("SELECT COUNT(DISTINCT name) FROM employee_positions;")
    count_nombres_unicos = cursor.fetchone()[0]
    print(f"  - Total de nombres únicos: {count_nombres_unicos}")

    # Para 'status'
    cursor.execute("SELECT DISTINCT status FROM employee_positions;")
    estados_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - status: {estados_unicos}")


except Exception as e:
    print(f"Ocurrió un error durante la verificación inicial de datos en 'employee_positions': {e}")


Total de registros en 'employee_positions': 15

Valores nulos por columna en 'employee_positions':
  - id: 0 nulos
  - name: 0 nulos
  - description: 0 nulos
  - salary: 0 nulos
  - status: 0 nulos
  - user_id: 0 nulos
  - created_at: 0 nulos
  - updated_at: 0 nulos

Filas duplicadas basadas en 'id' en 'employee_positions': 0

Valores únicos en columnas clave de 'employee_positions':
  - name (primeros 10 únicos): ['Desarrollador', 'General', 'Asistente Administrativo', 'Gerente', 'Tesorero principal', 'Contador General', 'Logístico Integral', 'Diseñador Web', 'Secretaria', 'Auxiliar de soporte']
  - Total de nombres únicos: 14
  - status: [1]


**Bloque 2: Limpieza y Transformación de la 1ra tabla: employee_positions**

In [62]:
# Script para la limpieza y transformación de 'employee_positions'
try:
    print("\nIniciando limpieza y transformación para 'employee_positions'...")

    # 1. Corrección de errores tipográficos en la columna 'name'
    # Detectamos 'Secretria' que debería ser 'Secretaria'
    print("Corrigiendo error tipográfico en la columna 'name': 'Secretria' a 'Secretaria'...")
    cursor.execute("""
        UPDATE employee_positions
        SET name = 'Secretaria'
        WHERE name = 'Secretria';
    """)
    conexion.commit() # Confirmar los cambios en la base de datos
    print("Corrección de 'name' completada.")

    # 2. Manejo de valores nulos en la columna 'description'
    # Rellenar con 'N/A' según lo acordado.
    print("Rellenando valores nulos en la columna 'description' con 'N/A'...")
    cursor.execute("""
        UPDATE employee_positions
        SET description = 'N/A'
        WHERE description IS NULL;
    """)
    conexion.commit() # Confirmar los cambios
    print("Relleno de 'description' completado.")

    print("\nLimpieza y transformación de 'employee_positions' finalizada.")

except Exception as e:
    print(f"Ocurrió un error durante la limpieza y transformación de 'employee_positions': {e}")
    conexion.rollback() # Revertir cambios en caso de error


Iniciando limpieza y transformación para 'employee_positions'...
Corrigiendo error tipográfico en la columna 'name': 'Secretria' a 'Secretaria'...
Corrección de 'name' completada.
Rellenando valores nulos en la columna 'description' con 'N/A'...
Relleno de 'description' completado.

Limpieza y transformación de 'employee_positions' finalizada.


**TABLA 2: EMPLOYEES o EMPLEADOS**


In [63]:
# Script para obtener la información de las columnas de 'employees'
try:
    # Obtener la descripción de las columnas de la tabla 'employees'
    cursor.execute("DESCRIBE employees;")
    columnas_employees = cursor.fetchall()

    print("Columnas de la tabla 'employees':")
    for columna in columnas_employees:
        # Cada 'columna' es una tupla con (nombre_columna, tipo_dato, ...)
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Ocurrió un error al intentar obtener las columnas de 'employees': {e}")

Columnas de la tabla 'employees':
  - id (int unsigned)
  - uuid (char(36))
  - identity_document_type (json)
  - document_number (varchar(25))
  - birth_date (date)
  - first_name (varchar(50))
  - other_name (varchar(50))
  - first_surname (varchar(50))
  - second_surname (varchar(50))
  - full_name (varchar(255))
  - phone (varchar(25))
  - email (varchar(75))
  - country (json)
  - region (json)
  - city (json)
  - address (varchar(255))
  - headquarter_id (int unsigned)
  - company_area_id (int unsigned)
  - employee_position_id (int unsigned)
  - emergency_contact (json)
  - payment_type (tinyint unsigned)
  - payment_method (json)
  - bank (json)
  - bank_account_type (json)
  - bank_account_number (varchar(255))
  - files (json)
  - status (tinyint(1))
  - settled (tinyint(1))
  - user_id (int unsigned)
  - created_at (timestamp)
  - updated_at (timestamp)
  - deleted_at (timestamp)


*Descripción de las variables*
id: int unsigned (Identificador único del empleado) - Clave primaria.
uuid: char(36) (Identificador universal único) - Probablemente para uso interno o integración.
identity_document_type: json (Tipo de documento de identidad) - ¡Atención! Es un JSON. Necesitaremos explorar su estructura interna.
document_number: varchar(25) (Número de documento)
birth_date: date (Fecha de nacimiento)
first_name: varchar(50) (Primer nombre)
other_name: varchar(50) (Otros nombres) - Puede ser nulo o vacío.
first_surname: varchar(50) (Primer apellido)
second_surname: varchar(50) (Segundo apellido) - Puede ser nulo o vacío.
full_name: varchar(255) (Nombre completo) - ¡Atención! Posible columna redundante si se puede construir a partir de first_name, other_name, first_surname, second_surname. Esto puede ser útil, pero también una fuente de inconsistencias si no se mantiene sincronizado.
phone: varchar(25) (Número de teléfono)
email: varchar(75) (Correo electrónico)
country: json (País) - ¡Atención! Otro JSON.
region: json (Región/Departamento/Estado) - ¡Atención! Otro JSON.
city: json (Ciudad) - ¡Atención! Otro JSON.
address: varchar(255) (Dirección completa)
headquarter_id: int unsigned (ID de la sede) - Clave foránea a la tabla headquarters.
company_area_id: int unsigned (ID del área de la compañía) - Clave foránea (¿a una tabla company_areas no listada inicialmente?).
employee_position_id: int unsigned (ID de la posición del empleado) - Clave foránea a employee_positions.
emergency_contact: json (Información de contacto de emergencia) - ¡Atención! Otro JSON.
payment_type: tinyint unsigned (Tipo de pago, e.g., fijo, por horas)
payment_method: json (Método de pago) - ¡Atención! Otro JSON.
bank: json (Información del banco) - ¡Atención! Otro JSON.
bank_account_type: json (Tipo de cuenta bancaria) - ¡Atención! Otro JSON.
user_id: int unsigned (ID del usuario que creó o modificó el registro)
created_at: timestamp (Marca de tiempo de creación del registro)
updated_at: timestamp (Marca de tiempo de la última actualización)
deleted_at: timestamp (Marca de tiempo de borrado lógico) - Indica si el registro ha sido "eliminado" lógicamente en lugar de físicamente.

Bloque 4: Verificación inicial de nulos, duplicados y valores únicos en employees


In [64]:
# Script para la limpieza y transformación inicial de 'employees' (VERSION FINAL)
print("\n--- Bloque 5: Limpieza y Transformación Inicial de 'employees' ---")

try:
    print("Iniciando operaciones de limpieza en 'employees'...")

    # 1. Manejo de valores nulos con 'N/A'
    print("Rellenando valores nulos en 'other_name', 'phone', 'email' con 'N/A'...")
    cursor.execute("""
        UPDATE employees
        SET other_name = 'N/A'
        WHERE other_name IS NULL;
    """)
    conexion.commit() # Confirmar los cambios

    cursor.execute("""
        UPDATE employees
        SET phone = 'N/A'
        WHERE phone IS NULL;
    """)
    conexion.commit() # Confirmar los cambios

    cursor.execute("""
        UPDATE employees
        SET email = 'N/A'
        WHERE email IS NULL;
    """)
    conexion.commit() # Confirmar los cambios
    print("Relleno de nulos completado.")

    # 2. Estandarización de capitalización para nombres
    # Aplicar CONCAT y SUBSTRING solo a los valores que NO sean 'N/A' o cadenas vacías
    print("Estandarizando capitalización de nombres...")

    # first_name
    cursor.execute("""
        UPDATE employees
        SET first_name = CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2)))
        WHERE first_name IS NOT NULL AND first_name != '';
    """)
    conexion.commit()

    # other_name (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET other_name = CONCAT(UPPER(SUBSTRING(other_name, 1, 1)), LOWER(SUBSTRING(other_name, 2)))
        WHERE other_name IS NOT NULL AND other_name != 'N/A' AND other_name != '';
    """)
    conexion.commit()

    # first_surname
    cursor.execute("""
        UPDATE employees
        SET first_surname = CONCAT(UPPER(SUBSTRING(first_surname, 1, 1)), LOWER(SUBSTRING(first_surname, 2)))
        WHERE first_surname IS NOT NULL AND first_surname != '';
    """)
    conexion.commit()

    # second_surname (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET second_surname = CONCAT(UPPER(SUBSTRING(second_surname, 1, 1)), LOWER(SUBSTRING(second_surname, 2)))
        WHERE second_surname IS NOT NULL AND second_surname != 'N/A' AND second_surname != '';
    """)
    conexion.commit()

    # full_name (solo si no es 'N/A' o vacío)
    cursor.execute("""
        UPDATE employees
        SET full_name = CONCAT(UPPER(SUBSTRING(full_name, 1, 1)), LOWER(SUBSTRING(full_name, 2)))
        WHERE full_name IS NOT NULL AND full_name != 'N/A' AND full_name != '';
    """)
    conexion.commit()

    print("Estandarización de capitalización completada.")

    print("\nLimpieza y transformación inicial de 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"Ocurrió un error inesperado durante la limpieza y transformación inicial de 'employees': {e}")
    # En caso de error, intentar limpiar el cursor y revertir.
    try:
        # Asegurar que cualquier resultado pendiente sea consumido antes de rollback
        if cursor.description is not None:
            cursor.fetchall()
        conexion.rollback()
        print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"Error adicional al intentar revertir la transacción o limpiar el cursor: {rollback_e}")


--- Bloque 5: Limpieza y Transformación Inicial de 'employees' ---
Iniciando operaciones de limpieza en 'employees'...
Rellenando valores nulos en 'other_name', 'phone', 'email' con 'N/A'...
Relleno de nulos completado.
Estandarizando capitalización de nombres...
Estandarización de capitalización completada.

Limpieza y transformación inicial de 'employees' finalizada exitosamente.


*Análisis realizado para la exploración en la segunda tabla y acciones a tomar* 
other_name (16 nulos)
email: 2 nulos.
phone: 3 nulos.
La muestra de identity_document_type ({"id": "13", "description": "Cédula de ciudadanía", "abbreviation": "C.C"}) nos da una idea clara de la estructura interna. Necesito "aplanar" o extraer esta información en columnas separadas para su uso.
Claves Foráneas (headquarter_id, company_area_id, employee_position_id)
Extracción y Aplanamiento de Columnas JSON
Limpieza y Transformación 
Capitalización estándar para nombres. 

In [65]:
# Script para limpiar y aplanar columnas JSON en 'employees' usando Pandas (ESTRATEGIA DEFINITIVA)

import pandas as pd
from db_connection import get_engine # Asegúrate de tener get_engine configurado

print("\n--- Bloque 6 (Estrategia Definitiva): Limpieza de JSONs en 'employees' con Pandas ---")

try:
    # 1. Obtener el engine de SQLAlchemy
    engine = get_engine()
    print("Engine de base de datos obtenido.")

    # 2. Cargar la tabla 'employees' completa en un DataFrame de Pandas
    print("Cargando la tabla 'employees' en un DataFrame de Pandas...")
    df_employees = pd.read_sql("SELECT * FROM employees;", engine)
    print(f"DataFrame de 'employees' cargado. Registros: {len(df_employees)}")

    # Lista de columnas JSON a procesar
    json_columns = [
        'identity_document_type',
        'country',
        'region',
        'city',
        'emergency_contact',
        'payment_method',
        'bank',
        'bank_account_type'
    ]

    # 3. Iterar sobre cada columna JSON y aplanarla en Pandas
    print("\nIniciando aplanamiento de columnas JSON en Pandas...")
    for col in json_columns:
        print(f"Procesando columna JSON: '{col}'...")
        
        # Función para extraer la descripción de un JSON en Python, manejando errores
        def extract_json_description(json_str):
            if pd.isna(json_str) or not isinstance(json_str, str):
                return 'N/A' # Manejar NaN o tipos no string
            try:
                # Intentar cargar como JSON. Si falla, es inválido.
                # json.loads() es más estricto y útil para capturar el error.
                import json
                data = json.loads(json_str)
                # Si es un objeto JSON y tiene 'description', la extraemos
                if isinstance(data, dict) and 'description' in data:
                    return str(data['description'])
                # Si es un objeto pero no tiene 'description', devolvemos 'N/A'
                return 'N/A'
            except json.JSONDecodeError:
                # Si no es un JSON válido o la carga falla, devolver 'N/A'
                print(f"    Advertencia: Valor inválido detectado en '{col}': '{json_str}'. Se reemplazará con 'N/A'.")
                return 'N/A'
            except Exception as e:
                # Otros errores inesperados durante el procesamiento del JSON
                print(f"    Advertencia: Error inesperado al procesar '{json_str}' en '{col}': {e}. Se reemplazará con 'N/A'.")
                return 'N/A'
        
        # Aplicar la función a la columna
        df_employees[col] = df_employees[col].apply(extract_json_description)
        print(f"  Columna '{col}' aplanada a 'description' o 'N/A'.")

    print("\nAplanamiento de JSONs en DataFrame completado.")

    # 4. Volver a guardar el DataFrame limpio en la base de datos
    # ¡ADVERTENCIA!: 'if_exists='replace'' BORRARÁ Y RECREARÁ LA TABLA.
    # Esta es la forma más directa de asegurar que los cambios se reflejen.
    print("Volcando el DataFrame limpio de vuelta a la tabla 'employees' en la base de datos...")
    df_employees.to_sql(name='employees', con=engine, if_exists='replace', index=False)
    print("DataFrame volcado a la base de datos exitosamente.")

    print("\nLimpieza y aplanamiento de JSONs en 'employees' finalizada exitosamente.")

except Exception as e:
    print(f"Ocurrió un error general durante el limpieza de JSONs en 'employees' con Pandas: {e}")
    print("Por favor, revisa la conexión a la base de datos o los datos en sí.")


--- Bloque 6 (Estrategia Definitiva): Limpieza de JSONs en 'employees' con Pandas ---
Engine de base de datos obtenido.
Cargando la tabla 'employees' en un DataFrame de Pandas...
DataFrame de 'employees' cargado. Registros: 38

Iniciando aplanamiento de columnas JSON en Pandas...
Procesando columna JSON: 'identity_document_type'...
  Columna 'identity_document_type' aplanada a 'description' o 'N/A'.
Procesando columna JSON: 'country'...
  Columna 'country' aplanada a 'description' o 'N/A'.
Procesando columna JSON: 'region'...
  Columna 'region' aplanada a 'description' o 'N/A'.
Procesando columna JSON: 'city'...
  Columna 'city' aplanada a 'description' o 'N/A'.
Procesando columna JSON: 'emergency_contact'...
  Columna 'emergency_contact' aplanada a 'description' o 'N/A'.
Procesando columna JSON: 'payment_method'...
  Columna 'payment_method' aplanada a 'description' o 'N/A'.
Procesando columna JSON: 'bank'...
  Columna 'bank' aplanada a 'description' o 'N/A'.
Procesando columna JSON:

*TABLA 3: employee_contracts o Contrato de empleados*


In [66]:
# Script para obtener la información de las columnas de 'employee_contracts' (RE-REVISADO 2)

# Asegúrate de que get_connection esté importado desde tu db_connection.py
from db_connection import get_connection

print("\n--- Bloque 7: Exploración de la Estructura de 'employee_contracts' ---")

# Reestablecer la conexión y el cursor
print("Reestableciendo conexión a la base de datos...")
try:
    # Cerrar si hay alguna conexión o cursor previo que no se cerró correctamente
    # SOLO la conexión tiene is_connected()
    if 'conexion' in globals() and conexion is not None and conexion.is_connected():
        # Primero cierra el cursor si existe y está asociado a la conexión
        if 'cursor' in globals() and cursor is not None:
            try:
                cursor.close()
                del cursor # Eliminar la variable del cursor
            except Exception as e:
                print(f"Advertencia al cerrar cursor previo: {e}")
        conexion.close()
        del conexion # Eliminar la variable de la conexión
    
    conexion = get_connection()
    cursor = conexion.cursor()
    print("Conexión y cursor reestablecidos.")
except Exception as e:
    print(f"ERROR: No se pudo reestablecer la conexión a la base de datos: {e}")
    # Si no podemos conectar, no tiene sentido continuar
    raise

try:
    cursor.execute("DESCRIBE employee_contracts;")
    columnas_employee_contracts = cursor.fetchall()

    print("Columnas de la tabla 'employee_contracts':")
    for columna in columnas_employee_contracts:
        print(f"  - {columna[0]} ({columna[1]})")

except Exception as e:
    print(f"Ocurrió un error al intentar obtener las columnas de 'employee_contracts': {e}")
finally:
    # Cerrar la conexión y el cursor al finalizar el bloque para limpiar recursos
    # Asegurarse de que las variables existan antes de intentar cerrarlas
    if 'cursor' in locals() and cursor is not None:
        try:
            cursor.close()
        except Exception as e:
            print(f"Advertencia al cerrar cursor en finally: {e}")
    if 'conexion' in locals() and conexion is not None and conexion.is_connected():
        try:
            conexion.close()
        except Exception as e:
            print(f"Advertencia al cerrar conexión en finally: {e}")
    print("Conexión cerrada al finalizar el bloque.")


--- Bloque 7: Exploración de la Estructura de 'employee_contracts' ---
Reestableciendo conexión a la base de datos...
✅ Conexión exitosa a la base de datos
Conexión y cursor reestablecidos.
Columnas de la tabla 'employee_contracts':
  - id (int unsigned)
  - employee_id (int unsigned)
  - employee_type (varchar(255))
  - employee_subtype (varchar(255))
  - high_penson_risk (tinyint(1))
  - integral_salary (tinyint(1))
  - type (varchar(255))
  - start_date (date)
  - end_date (date)
  - salary (double(16,2))
  - transport_subsidy (tinyint(1))
  - risk_type (varchar(255))
  - holidays_enjoyed (smallint unsigned)
  - health_provider_id (bigint unsigned)
  - health_provider (varchar(255))
  - pension_provider (varchar(255))
  - pension_provider_id (bigint unsigned)
  - severance_provider (varchar(255))
  - severance_provider_id (bigint unsigned)
  - family_compensation_fund_provider (json)
  - family_compensation_fund_provider_id (bigint unsigned)
  - settlement_cause (varchar(255))
  - 

*Descripción de las tablas:* 
id: int unsigned (Identificador único del contrato)
employee_id: int unsigned (ID del empleado al que pertenece el contrato) - Clave Foránea a la tabla employees.
employee_type: json (Tipo de empleado, e.g., fijo, temporal) - JSON a aplanar.
employee_subtype: json (Subtipo de empleado, si aplica) - JSON a aplanar.
high_penson_risk: tinyint(1) (Indicador de alto riesgo de pensión, booleano)
integral_salary: tinyint(1) (Indicador de salario integral, booleano)
type: json (Tipo de contrato, e.g., a término indefinido, obra labor) - JSON a aplanar.
start_date: date (Fecha de inicio del contrato)
end_date: date (Fecha de fin del contrato) - ¡Atención! Puede ser nulo para contratos indefinidos o activos.
salary: double(16,2) (Salario del contrato)
transport_subsidy: tinyint(1) (Indicador de subsidio de transporte, booleano)
risk_type: json (Tipo de riesgo laboral) - JSON a aplanar.
holidays_enjoyed: smallint unsigned (Días de vacaciones disfrutados)
health_provider_id: bigint unsigned (ID del proveedor de salud) - Posible clave foránea.
health_provider: json (Información del proveedor de salud) - JSON a aplanar.
pension_provider: json (Información del proveedor de pensión) - JSON a aplanar.
pension_provider_id: bigint unsigned (ID del proveedor de pensión) - Posible clave foránea.
severance_provider: json (Información del proveedor de cesantías) - JSON a aplanar.
severance_provider_id: bigint unsigned (ID del proveedor de cesantías) - Posible clave foránea.
settlement_approved_at: timestamp (Fecha de aprobación de liquidación) - ¡Atención! Probablemente nulo para contratos activos.
created_at: timestamp (Marca de tiempo de creación del registro)
updated_at: timestamp (Marca de tiempo de la última actualización)

In [67]:
# Script para la verificación inicial de datos en 'employee_contracts'

# Asegúrate de que get_connection esté importado desde tu db_connection.py
from db_connection import get_connection

print("\n--- Bloque 8: Verificación Inicial de 'employee_contracts' ---")

# Reestablecer la conexión y el cursor para este bloque
print("Reestableciendo conexión a la base de datos...")
try:
    if 'conexion' in globals() and conexion is not None and conexion.is_connected():
        if 'cursor' in globals() and cursor is not None:
            try:
                cursor.close()
                del cursor
            except Exception as e:
                print(f"Advertencia al cerrar cursor previo: {e}")
        conexion.close()
        del conexion
    
    conexion = get_connection()
    cursor = conexion.cursor()
    print("Conexión y cursor reestablecidos.")
except Exception as e:
    print(f"ERROR: No se pudo reestablecer la conexión a la base de datos: {e}")
    raise

try:
    # 1. Obtener el número total de registros
    cursor.execute("SELECT COUNT(*) FROM employee_contracts;")
    total_registros_contracts = cursor.fetchone()[0]
    print(f"\nTotal de registros en 'employee_contracts': {total_registros_contracts}")

    # 2. Verificar valores nulos por columna
    print("\nValores nulos por columna en 'employee_contracts':")
    # Para esto, usaremos la lista de columnas que acabas de compartir
    # (asumiendo que 'columnas_employee_contracts' está disponible o se recrea)
    cursor.execute("DESCRIBE employee_contracts;")
    columnas_employee_contracts = cursor.fetchall()
    
    for columna_info in columnas_employee_contracts:
        nombre_columna = columna_info[0]
        cursor.execute(f"SELECT COUNT(*) FROM employee_contracts WHERE {nombre_columna} IS NULL;")
        nulos_count = cursor.fetchone()[0]
        if nulos_count > 0:
            print(f"  - {nombre_columna}: {nulos_count} nulos")
        else:
            print(f"  - {nombre_columna}: 0 nulos")

    # 3. Verificar filas duplicadas
    cursor.execute("SELECT COUNT(id) - COUNT(DISTINCT id) FROM employee_contracts;")
    duplicados_id_contracts = cursor.fetchone()[0]
    print(f"\nFilas duplicadas basadas en 'id' en 'employee_contracts': {duplicados_id_contracts}")

    # Es posible que un empleado tenga múltiples contratos a lo largo del tiempo,
    # pero no debería tener dos contratos con la misma fecha de inicio.
    cursor.execute("""
        SELECT COUNT(*)
        FROM (
            SELECT employee_id, start_date, COUNT(*)
            FROM employee_contracts
            GROUP BY employee_id, start_date
            HAVING COUNT(*) > 1
        ) AS duplicated_contracts;
    """)
    duplicados_employee_date = cursor.fetchone()[0]
    print(f"Contratos duplicados basados en 'employee_id' y 'start_date': {duplicados_employee_date}")

    # 4. Obtener valores únicos para columnas booleanas (tinyint) y una muestra de JSON
    print("\nValores únicos en columnas clave de 'employee_contracts':")

    # Para 'high_penson_risk'
    cursor.execute("SELECT DISTINCT high_penson_risk FROM employee_contracts;")
    high_penson_risk_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - high_penson_risk: {high_penson_risk_unicos}")

    # Para 'integral_salary'
    cursor.execute("SELECT DISTINCT integral_salary FROM employee_contracts;")
    integral_salary_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - integral_salary: {integral_salary_unicos}")

    # Para 'transport_subsidy'
    cursor.execute("SELECT DISTINCT transport_subsidy FROM employee_contracts;")
    transport_subsidy_unicos = [fila[0] for fila in cursor.fetchall()]
    print(f"  - transport_subsidy: {transport_subsidy_unicos}")

    # 5. Muestra de contenido de una columna JSON: 'employee_type' (primeros 5)
    print("\nMuestra de contenido de la columna JSON 'employee_type' (primeros 5):")
    cursor.execute("SELECT employee_type FROM employee_contracts WHERE employee_type IS NOT NULL LIMIT 5;")
    employee_type_sample = [fila[0] for fila in cursor.fetchall()]
    for i, etype in enumerate(employee_type_sample):
        print(f"  - Muestra {i+1}: {etype}")

except Exception as e:
    print(f"Ocurrió un error durante la verificación inicial de datos en 'employee_contracts': {e}")
finally:
    # Cerrar la conexión al finalizar el bloque para limpiar recursos
    if 'cursor' in locals() and cursor is not None:
        try:
            cursor.close()
        except Exception as e:
            print(f"Advertencia al cerrar cursor en finally: {e}")
    if 'conexion' in locals() and conexion is not None and conexion.is_connected():
        try:
            conexion.close()
        except Exception as e:
            print(f"Advertencia al cerrar conexión en finally: {e}")
    print("Conexión cerrada al finalizar el bloque.")


--- Bloque 8: Verificación Inicial de 'employee_contracts' ---
Reestableciendo conexión a la base de datos...
✅ Conexión exitosa a la base de datos
Conexión y cursor reestablecidos.

Total de registros en 'employee_contracts': 43

Valores nulos por columna en 'employee_contracts':
  - id: 0 nulos
  - employee_id: 0 nulos
  - employee_type: 0 nulos
  - employee_subtype: 0 nulos
  - high_penson_risk: 0 nulos
  - integral_salary: 0 nulos
  - type: 0 nulos
  - start_date: 0 nulos
  - end_date: 32 nulos
  - salary: 0 nulos
  - transport_subsidy: 0 nulos
  - risk_type: 0 nulos
  - holidays_enjoyed: 0 nulos
  - health_provider_id: 5 nulos
  - health_provider: 0 nulos
  - pension_provider: 2 nulos
  - pension_provider_id: 5 nulos
  - severance_provider: 2 nulos
  - severance_provider_id: 5 nulos
  - family_compensation_fund_provider: 0 nulos
  - family_compensation_fund_provider_id: 5 nulos
  - settlement_cause: 30 nulos
  - settlement_with_just_cause: 0 nulos
  - settlement_probationary_peri

#Resultado análisis y toma de acciones: 
health_provider_id: 5 nulos. Rellenaremos con N/A.
pension_provider: 2 nulos. Rellenaremos con N/A.
pension_provider_id: 2 nulos. Rellenaremos con N/A.
severance_provider: 2 nulos. Rellenaremos con N/A.
severance_provider_id: 2 nulos. Rellenaremos con N/A.
settlement_approved_at: 43 nulos. Esto significa que ningún contrato ha sido liquidado o aprobado para liquidación. Es importante mantener esto en cuenta para el análisis futuro (e.g., todos los contratos son activos o no han llegado a su fase de liquidación).
Aplanamiento de Columnas JSON
Limpieza y Transformación

In [79]:
# Script para limpiar y aplanar 'employee_contracts' directamente
import pandas as pd
import json
import numpy as np # Asegúrate de que numpy esté importado
from db_connection import get_engine, get_connection 

print("\n--- Bloque 9: Limpieza y Transformación de 'employee_contracts' ---")

# --- PASO CRÍTICO: CERRAR Y REABRIR CONEXIÓN PARA ASEGURAR ESTADO LIMPIO ---
print("Reestableciendo conexión a la base de datos...")
conexion = None 
cursor = None 
try:
    if 'conexion' in globals() and conexion is not None and conexion.is_connected():
        if 'cursor' in globals() and cursor is not None:
            try:
                cursor.close()
            except Exception as e:
                print(f"Advertencia al cerrar cursor previo: {e}")
        conexion.close()
    
    conexion = get_connection()
    cursor = conexion.cursor()
    print("✅ Conexión y cursor reestablecidos.")
except Exception as e:
    print(f"❌ ERROR: No se pudo reestablecer la conexión a la base de datos: {e}")
    raise 

try:
    engine = get_engine()
    print("✅ Engine de base de datos obtenido.")

    # --- PASO 1: Cargando la tabla 'employee_contracts' en Pandas y realizando transformaciones ---
    print("\n--- PASO 1: Cargando 'employee_contracts' en Pandas y realizando transformaciones ---")
    df_contracts = pd.read_sql("SELECT * FROM employee_contracts;", engine)
    print(f"DataFrame de contratos cargado desde 'employee_contracts'. Registros: {len(df_contracts)}")

    # 1.1. Manejo de valores nulos y asegurando nombres de columnas válidos
    print("Rellenando valores nulos para columnas específicas con 'N/A' y asegurando nombres válidos...")
    cols_to_fill_na = [
        'health_provider_id', 
        'pension_provider',
        'pension_provider_id',
        'severance_provider',
        'severance_provider_id'
    ]
    for col in cols_to_fill_na:
        if col in df_contracts.columns:
            if 'id' not in col:
                df_contracts[col] = df_contracts[col].astype(str).replace('nan', 'N/A').fillna('N/A')
            else:
                df_contracts[col] = df_contracts[col].replace({np.nan: None})
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame para rellenar nulos.")
    
    df_contracts.columns = [str(col) if pd.notna(col) else f"unnamed_col_{i}" for i, col in enumerate(df_contracts.columns)]
    
    print("✅ Nombres de columnas del DataFrame validados.")
    print("✅ Relleno de nulos completado en DataFrame.")

    # 1.2. Aplanamiento de Columnas JSON
    print("\nIniciando aplanamiento de columnas JSON en Pandas...")
    json_columns = [
        'employee_type',
        'employee_subtype',
        'type',
        'risk_type',
        'health_provider',
        'pension_provider',
        'severance_provider',
        # 'family_compensation_fund_provider' <-- Sigue excluida como acordamos
    ]

    def extract_json_description_or_code(json_str):
        if pd.isna(json_str) or not isinstance(json_str, str) or not json_str.strip().startswith('{'):
            return None 
        try:
            data = json.loads(json_str)
            if isinstance(data, dict):
                if 'description' in data:
                    return str(data['description'])
                elif 'code' in data: 
                    return str(data['code'])
            return None 
        except json.JSONDecodeError:
            return None 
        except Exception as e:
            print(f"   Error inesperado al procesar JSON: {e} en '{str(json_str)[:50]}'")
            return None

    for col in json_columns:
        if col in df_contracts.columns:
            df_contracts[col] = df_contracts[col].astype(str)
            df_contracts[col] = df_contracts[col].apply(extract_json_description_or_code)
            print(f"   Columna '{col}' aplanada a 'description'/'code' o NULL.")
        else:
            print(f"   Advertencia: Columna '{col}' no encontrada en el DataFrame para aplanar JSON.")
    print("✅ Aplanamiento de JSONs en DataFrame completado.")
    
    # --- PASO 2: ELIMINADO - NO SE ALTERA LA ESTRUCTURA DE LA BASE DE DATOS ---

    # --- PASO 3: Actualizar los registros con los datos limpios del DataFrame ---
    print("\n--- PASO 3: Actualizando registros en 'employee_contracts' con datos limpios ---")
    
    cursor.execute("DESCRIBE `employee_contracts`;")
    db_columns_info = cursor.fetchall()
    db_column_names = [col_info[0] for col_info in db_columns_info]

    valid_columns_for_update = [
        col for col in df_contracts.columns
        if isinstance(col, str) and col != 'id' and col in db_column_names
    ]
    
    print(f"\nDEBUG: Columnas en el DataFrame: {df_contracts.columns.tolist()}")
    print(f"DEBUG: Columnas en la Base de Datos: {db_column_names}")
    print(f"DEBUG: Columnas que se usarán para UPDATE: {valid_columns_for_update}")

    if not valid_columns_for_update:
        print("⚠️ Advertencia: No hay columnas válidas para actualizar (aparte del 'id'). No se realizará ninguna actualización.")
    else:
        set_clauses = [f"`{col}` = %s" for col in valid_columns_for_update]
        update_sql = f"UPDATE `employee_contracts` SET {', '.join(set_clauses)} WHERE `id` = %s;"
        
        print(f"DEBUG: SQL de UPDATE generado: {update_sql[:150]}...")

        updated_count = 0
        for index, row in df_contracts.iterrows():
            update_values = []
            for col in valid_columns_for_update:
                val = row[col]
                # Conversión final de valores de Pandas/NumPy a tipos compatibles con MySQL
                # Corregido: Reemplazar np.float_ con np.float64 para compatibilidad con NumPy 2.0+
                if pd.isna(val) or (isinstance(val, (float, np.float64)) and np.isnan(val)):
                    update_values.append(None) 
                elif isinstance(val, (np.int64, np.int32)):
                    update_values.append(int(val)) 
                elif isinstance(val, (np.bool_)):
                    update_values.append(bool(val)) 
                else:
                    update_values.append(val)
                    
            update_values.append(row['id']) 
            
            try:
                cursor.execute(update_sql, tuple(update_values))
                updated_count += 1
                if updated_count % 10 == 0:
                    print(f"   Progreso: {updated_count}/{len(df_contracts)} registros procesados...")
            except Exception as e:
                print(f"❌ Error al actualizar el registro con ID {row['id']}: {e}")
                print(f"   Valores que intentaron insertarse para ID {row['id']}: {tuple(update_values)}")
                conexion.rollback()
                raise 

        conexion.commit() 
        print(f"✅ Actualización de {updated_count} registros en la tabla 'employee_contracts' completada exitosamente.")
    
    print("\n✅ Limpieza y transformación de 'employee_contracts' finalizada exitosamente.")

except Exception as e:
    print(f"❌ Ocurrió un error general durante la limpieza de 'employee_contracts': {e}")
    try:
        if 'conexion' in locals() and conexion is not None and conexion.is_connected():
            conexion.rollback()
            print("Transacción revertida debido a un error.")
    except Exception as rollback_e:
        print(f"❌ Error adicional al intentar revertir: {rollback_e}")
    
finally:
    print("Cerrando la conexión después del procesamiento.")
    try:
        if 'cursor' in locals() and cursor is not None:
            cursor.close()
        if 'conexion' in locals() and conexion is not None and conexion.is_connected():
            conexion.close()
        print("✅ Conexión cerrada exitosamente.")
    except Exception as e:
        print(f"❌ Advertencia al cerrar la conexión en finally: {e}")


--- Bloque 9: Limpieza y Transformación de 'employee_contracts' ---
Reestableciendo conexión a la base de datos...
✅ Conexión exitosa a la base de datos
✅ Conexión y cursor reestablecidos.
✅ Engine de base de datos obtenido.

--- PASO 1: Cargando 'employee_contracts' en Pandas y realizando transformaciones ---
DataFrame de contratos cargado desde 'employee_contracts'. Registros: 43
Rellenando valores nulos para columnas específicas con 'N/A' y asegurando nombres válidos...
✅ Nombres de columnas del DataFrame validados.
✅ Relleno de nulos completado en DataFrame.

Iniciando aplanamiento de columnas JSON en Pandas...
   Columna 'employee_type' aplanada a 'description'/'code' o NULL.
   Columna 'employee_subtype' aplanada a 'description'/'code' o NULL.
   Columna 'type' aplanada a 'description'/'code' o NULL.
   Columna 'risk_type' aplanada a 'description'/'code' o NULL.
   Columna 'health_provider' aplanada a 'description'/'code' o NULL.
   Columna 'pension_provider' aplanada a 'descrip

In [ ]:
# Cerrar conexión
cursor.close()
conexion.close()
